<a href="https://colab.research.google.com/github/adimnicky/Capstone-Project/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community
!pip install replicate
!pip install pandas

In [ ]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata
import pandas as pd

# Set the API token
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token
# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
model=model,
replicate_api_token=api_token,
)

In [22]:
# Load dataset from CSV
df = pd.read_csv("Tweets.csv")

# Define a function to split the list into batches
def batch_list(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

# Define the batch size (adjust as needed based on the model's context window)
batch_size = 100 # You can adjust this number

# Create batches of reviews
review_batches = list(batch_list(customer_reviews_dataset, batch_size))

# Process each batch and store the responses
responses = []
for i, batch in enumerate(review_batches):
    # Use the prompt_template defined earlier
    reviews_text_batch = "\n".join([f"Review {j+1}: {review}" for j, review in enumerate(batch)])
    prompt_batch = prompt_template.format(reviews_text_batch=reviews_text_batch)

    print(f"Processing batch {i+1}/{len(review_batches)}...")
    response_batch = output.invoke(prompt_batch)
    responses.append(response_batch)

# Combine the responses (this will depend on the desired output format)
# For now, let's just print each response
for i, response in enumerate(responses):
    print(f"\nResponse for batch {i+1}:\n")
    print(response)

Processing batch 1/147...
Processing batch 2/147...
Processing batch 3/147...
Processing batch 4/147...
Processing batch 5/147...
Processing batch 6/147...
Processing batch 7/147...
Processing batch 8/147...
Processing batch 9/147...
Processing batch 10/147...
Processing batch 11/147...
Processing batch 12/147...
Processing batch 13/147...
Processing batch 14/147...
Processing batch 15/147...
Processing batch 16/147...
Processing batch 17/147...
Processing batch 18/147...
Processing batch 19/147...
Processing batch 20/147...
Processing batch 21/147...
Processing batch 22/147...
Processing batch 23/147...
Processing batch 24/147...
Processing batch 25/147...
Processing batch 26/147...
Processing batch 27/147...
Processing batch 28/147...
Processing batch 29/147...
Processing batch 30/147...
Processing batch 31/147...
Processing batch 32/147...
Processing batch 33/147...
Processing batch 34/147...
Processing batch 35/147...
Processing batch 36/147...
Processing batch 37/147...
Processing

In [26]:
# Define refined prompt
refined_prompt = f"""
Classify these reviews as positive, negative, or mixed, and tag
relevant categories (late flight, or rude service):
{response_batch}
"""
# Invoke the model with the example prompt
response = output.invoke(refined_prompt)
# Print the response
print("Granite Model Refined Response:\n")
print(response)

Granite Model Refined Response:

Based on the provided analysis, here's a structured classification of the reviews:

**Positive Reviews:**
- **Review 20**: Positive - Expresses gratitude despite past issues, highlighting satisfactory resolution and service.
- **Review 6**: Mixed (leaning Positive) - Initially expresses frustration but later thanks AA for refund and assistance.
- **Review 28**: Positive - User is content with the service received.
- **Review 36**: Positive - User praises the staff and service.
- **Review 39**: Mixed (leaning Positive) - User is frustrated but acknowledges AA's handling of their money matter, seeking resolution.

**Negative Reviews:**
- **Review 1**: Negative - Complains about flight cancellation and poor communication.
- **Review 4**: Negative - Criticizes rude staff and lack of assistance.
- **Review 5**: Negative - Dissatisfied with flight delays and poor customer service.
- **Review 7**: Negative - Unhappy with flight cancellation and rebooking issue

In [27]:
# Define the prompt to complete the task in 2 steps
multitask_prompt = f"""
Complete the task in 2 steps.
Step 1: Classify these reviews as positive, negative, or mixed.
Step 2: For each review, identify relevant categories: battery
life, screen quality, or performance.
{response_batch}
"""
response = output.invoke(multitask_prompt)
print("Granite Model Response:\n")
print(response)

Granite Model Response:

### Step 1: Classification

**Positive Reviews:**
- Review 20: Expresses gratitude for assistance received, indicating a positive experience post-resolution.
- Review 6: Despite initial frustration, the user eventually thanks AA, leaning positive.
- Review 28: Clearly positive, praising service or experience.
- Review 36: Positive feedback acknowledging good service.
- Review 39: Although expressing frustration, the user acknowledges AA's handling of their money, suggesting a mixed-to-positive stance.

**Negative Reviews:**
- Reviews 1, 4, 5, 7, 9, 10, 11, 14, 17, 21, 22, 24, 27, 32, 34, 37: All directly criticize flight cancellations, delays, poor communication, or rudeness of staff, clearly negative.

**Mixed Reviews:**
- Review 3: Seeks assistance, indicating ongoing issues but not purely negative.
- Review 8: Expresses some dissatisfaction but also acknowledges positive aspects, mixed sentiment.
- Review 12: Mixed feelings due to acknowledging both positive

In [28]:
# Define the example to guide the model
formatted_prompt = f"""
Classify these reviews as Positive, Negative, or Mixed, and tag
relevant categories. Use this format:
- Sentiment: [Sentiment]
- Categories: [Categories].
{response_batch}
"""
# Invoke the model with prompt
response = output.invoke(formatted_prompt)
# Print the response
print("Granite Model Formatted Response:\n")
print(response)

Granite Model Formatted Response:

- Sentiment:

**Positive:**
- Review 20: Sentiment is positive as the customer expresses gratitude for a resolved issue and satisfaction with the resolution process.
- Categories: Customer service resolution, problem-solving.

- Review 6: Initially negative due to frustration but shifts to positive acknowledging American Airlines' handling of their money refund.
- Categories: Refund process, customer service recovery.

- Review 28: Positive, customer appreciates the legroom and overall flight experience.
- Categories: In-flight comfort, service.

- Review 36: Positive, customer praises the crew and flight experience.
- Categories: Cabin crew, flight experience.

- Review 39: Despite expressing frustration, the customer seeks assistance and acknowledges having their money, indicating a positive outlook on resolution.
- Categories: Customer service engagement, financial resolution.

**Negative:**
- Reviews 1, 4, 5, 7, 9, 10, 11, 14, 17, 21, 22, 24, 27, 